Now that we know that our best model is made with the VGG16 pretrained model we can use a new file

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Hyperparameters
IMAGE_SIZE = 256
BATCH_SIZE = 8
CHANNELS= 3
ES_EPOCHS = 10 
CLASSES = 10
EPOCHS= 200

In [4]:
CLASS_NAMES = ["1989", "Acoustic", "Fearless", "Folkmore", "Lover", "Midnights", "Red", "Reputation", "Speak Now", "TTPD"]

Create the data splits

In [5]:
import os
import shutil
import random

def split_dataset(folder_path, output_path, train_split=0.8, test_split=0.1, 
                   val_split=0.1, shuffle=True):
    """
    Splits the images in the given folder into train, test, and validation sets,
    and moves them to the specified output path, ensuring no duplicate moves.

    Args:
        folder_path (str): Path to the folder containing the images.
        output_path (str): Path to the directory where the split datasets 
                          will be saved.
        train_split (float, optional): Ratio of images for training (0.0 to 1.0). 
                                     Defaults to 0.8.
        test_split (float, optional): Ratio of images for testing (0.0 to 1.0). 
                                    Defaults to 0.1.
        val_split (float, optional): Ratio of images for validation (0.0 to 1.0). 
                                    Defaults to 0.1.
        shuffle (bool, optional): Whether to shuffle the images. Defaults to True.
        shuffle_size (int, optional): Size of the shuffle buffer. Defaults to 1000.
    """

    if train_split + test_split + val_split != 1.0:
        raise ValueError("Train, test, and validation splits must add up to 1.0")

    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Create subfolders for train, test, and validation in the output directory
    for subfolder in ["train", "test", "val"]:
        os.makedirs(os.path.join(output_path, subfolder), exist_ok=True)

    # Get list of image files in the folder
    image_files = [
        f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))
    ]

    # Shuffle the image files
    if shuffle:
        random.shuffle(image_files)

    # Calculate the number of images for each set
    num_images = len(image_files)
    num_train = int(num_images * train_split)
    num_test = int(num_images * test_split)

    # Split the image files into sets
    train_files = image_files[:num_train]
    test_files = image_files[num_train : num_train + num_test]
    val_files = image_files[num_train + num_test :]

    # Move images to corresponding subfolders, preventing duplicates
    for file_list, subfolder in zip(
        [train_files, test_files, val_files], ["train", "test", "val"]
    ):
        for image_file in file_list:
            src_path = os.path.join(folder_path, image_file)
            dest_path = os.path.join(output_path, subfolder, image_file)
            if not os.path.exists(dest_path):  # Check if file already exists
                shutil.move(src_path, dest_path) 
 

Create the classes

In [6]:
def organize_images(directory):
  """Organizes images in a directory into subfolders based on the first word of their filenames.

  Args:
    directory: The path to the directory containing the images.
  """

  for filename in os.listdir(directory):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.gif')):  # Adjust file extensions if needed
      file_path = os.path.join(directory, filename)
      folder_name = filename.split('_')[0]  # Extract the first word before "_"
      folder_path = os.path.join(directory, folder_name)

      # Create the folder if it doesn't exist
      if not os.path.exists(folder_path):
        os.makedirs(folder_path)

      # Move the image to the corresponding folder
      shutil.move(file_path, os.path.join(folder_path, filename))

In [7]:
output_path = "C:\\Users\\Ronan\\Documents\\ML\\Taylor_Swift_Projects\\CNN\\"

# 1989
_1989_path = "C:\\Users\\Ronan\\Documents\\ML\\Taylor_Swift_Projects\\CNN\\Latest_TS_Dataset\\1989"
split_dataset(_1989_path, output_path) 

# Acoustic
acoustic_path = "C:\\Users\\Ronan\\Documents\\ML\\Taylor_Swift_Projects\\CNN\\Latest_TS_Dataset\\Acoustic"
split_dataset(acoustic_path, output_path) 

# Fearless
fearless_path = "C:\\Users\\Ronan\\Documents\\ML\\Taylor_Swift_Projects\\CNN\\Latest_TS_Dataset\\Fearless"
split_dataset(fearless_path, output_path)

# Folkmore
folkmore_path = "C:\\Users\\Ronan\\Documents\\ML\\Taylor_Swift_Projects\\CNN\\Latest_TS_Dataset\\Folkmore"
split_dataset(folkmore_path, output_path)

# Lover
lover_path = "C:\\Users\\Ronan\\Documents\\ML\\Taylor_Swift_Projects\\CNN\\Latest_TS_Dataset\\Lover"
split_dataset(lover_path, output_path)

# Midnights
midnights_path = "C:\\Users\\Ronan\\Documents\\ML\\Taylor_Swift_Projects\\CNN\\Latest_TS_Dataset\\Midnights"
split_dataset(midnights_path, output_path)

# Red
red_path = "C:\\Users\\Ronan\\Documents\\ML\\Taylor_Swift_Projects\\CNN\\Latest_TS_Dataset\\Red"
split_dataset(red_path, output_path)

# Reputation
reputation_path = "C:\\Users\\Ronan\\Documents\\ML\\Taylor_Swift_Projects\\CNN\\Latest_TS_Dataset\\Reputation"
split_dataset(reputation_path, output_path)

# Speak Now
speak_now_path = "C:\\Users\\Ronan\\Documents\\ML\\Taylor_Swift_Projects\\CNN\\Latest_TS_Dataset\\Speak Now"
split_dataset(speak_now_path, output_path)

# TTPD
ttpd_path = "C:\\Users\\Ronan\\Documents\\ML\\Taylor_Swift_Projects\\CNN\\Latest_TS_Dataset\\TTPD"
split_dataset(ttpd_path, output_path)

In [8]:
# Train
train_directory = "C:/Users/Ronan/Documents/ML/Taylor_Swift_Projects/CNN/train"
organize_images(train_directory)

# Val
val_directory = "C:/Users/Ronan/Documents/ML/Taylor_Swift_Projects/CNN/val"
organize_images(val_directory)

# Test
test_directory = "C:/Users/Ronan/Documents/ML/Taylor_Swift_Projects/CNN/test"
organize_images(test_directory)

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable

In [ ]:
# Directory with the images
test_dir = "C:/Users/Ronan/Documents/ML/Taylor_Swift_Projects/CNN/test"
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    seed=42,
    label_mode='int',
    validation_split=None,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

In [ ]:
# Directory with the images
train_dir = "C:/Users/Ronan/Documents/ML/Taylor_Swift_Projects/CNN/train"
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    seed = 42,
    label_mode='int',
    validation_split=None,
    shuffle = True,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

In [ ]:
# Directory with the images
val_dir = "C:/Users/Ronan/Documents/ML/Taylor_Swift_Projects/CNN/val"
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    labels='inferred',
    seed = 42,
    label_mode='int',
    validation_split=None,
    shuffle = True,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

In [ ]:
print(f"""
    ----------------------------------
    Dataset split:
        - train split: {len(train_ds)}
        - val split: {len(val_ds)}
        - test split: {len(test_ds)}
    ----------------------------------
    """
)

In [ ]:
#Caching the dataset

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Normalize and rescale the images
#resize_and_rescale = tf.keras.Sequential([
#  tf.keras.layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
#  tf.keras.layers.Rescaling(1./255),
#])

In [ ]:
# Proof that the images are rescaled
#normalized_ds = train_ds.map(lambda x, y: (resize_and_rescale(x), y))
#image_batch, labels_batch = next(iter(normalized_ds))
#first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
#print(np.min(first_image), np.max(first_image))

In [ ]:
# Import keras api 
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.1),
  tf.keras.layers.RandomContrast(0.2),
  tf.keras.layers.RandomZoom(0.2),
  tf.keras.layers.RandomBrightness(factor=0.2)
])

In [ ]:
#train_ds = train_ds.map(
#    lambda x, y: (data_augmentation(x, training=True), y)
#).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
import os
import seaborn as sns
main_folder = 'train'

In [ ]:
era_counts = {}

# Iterate through each subfolder (era)
for era_folder in os.listdir(main_folder):
    era_path = os.path.join(main_folder, era_folder)
    if os.path.isdir(era_path):
        # Count the number of images in the era folder
        era_counts[era_folder] = len([filename for filename in os.listdir(era_path) if filename.lower().endswith(('.jpg', '.jpeg', '.png'))])

# Create a pandas DataFrame from the era_counts dictionary
import pandas as pd
df = pd.DataFrame({'Era': era_counts.keys(), 'Image Count': era_counts.values()})

# Create the bar plot using seaborn
plt.figure(figsize=(10, 6))  # Adjust figure size as needed
ax = sns.barplot(x='Era', y='Image Count', data=df, palette='flare')  # Choose a color palette you like
for bar in range(len(ax.containers)):
    ax.bar_label(ax.containers[bar])
# Customize the plot
plt.title('Data Imbalance in Taylor Swift Eras Tour Images', fontsize=16)
plt.xlabel('Eras', fontsize=14)
plt.ylabel('Number of Images', fontsize=14)
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.tight_layout()  # Adjust layout for better spacing

# Display the plot
plt.show()

In [ ]:
def get_weight_inverse_num_samples(num_of_classes, samples_per_classes, coeff=1):
    """
    Get the inverse number of samples for each class.
    Args:
        num_of_classes (int): Number of classes in the dataset.
        samples_per_classes (list): List of number of samples per class.
        power (int, optional): Power of the inverse number of samples. Defaults to 1.
    Returns:
        list: List of inverse number of samples for each class.
    """

    inverse_num_samples = []
    total_samples = sum(samples_per_classes)

    for i in range(num_of_classes):
        inverse_num_samples.append((total_samples / (num_of_classes * samples_per_classes[i])) * coeff)

    return inverse_num_samples

num_of_classes = 10
samples_per_classes = [era_counts['1989'], era_counts['Acoustic'], era_counts['Fearless'], era_counts['Folkmore'], era_counts['Lover'], era_counts['Midnights'], era_counts['Red'], era_counts['Reputation'], era_counts['Speak'], era_counts['TTPD']] 

weighted_classes = get_weight_inverse_num_samples(num_of_classes, samples_per_classes)


In [ ]:
class_indexes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
weights = dict(zip(class_indexes, weighted_classes))

In [ ]:
weights

Use pretrained model ResNet50 to experiment if we can have further improvements

In [ ]:
conv_base = tf.keras.applications.ResNet50(
weights="imagenet",
include_top=False,
input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS))

Used pretrained model VGG16

In [ ]:
#conv_base = tf.keras.applications.vgg16.VGG16(
#weights="imagenet",
#include_top=False,
#input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS))

In [ ]:
for layer in conv_base.layers[:-12]:
    layer.trainable = False

In [ ]:
conv_base.summary()

In [ ]:
from tensorflow.keras.regularizers import l2

In [ ]:
inputs = tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS))

x = data_augmentation(inputs)

#x = resize_and_rescale(x)

x = tf.keras.applications.resnet.preprocess_input(x)
#x = tf.keras.applications.vgg16.preprocess_input(x)

x = conv_base(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256)(x)
x = tf.keras.layers.Dropout(0.5)(x)

outputs = tf.keras.layers.Dense(CLASSES, activation="softmax", kernel_regularizer=l2(0.001))(x)
#outputs = tf.keras.layers.Dense(CLASSES, activation="softmax")(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras import callbacks

In [ ]:
filepath="saved_models/all_VGG16_models/best_model_weighted_RESNET50.keras"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
lr_plateau = callbacks.ReduceLROnPlateau(factor=0.8, monitor="val_accuracy", patience=5, verbose=1)
es = EarlyStopping(monitor='val_accuracy', patience=ES_EPOCHS)
callbacks_list = [checkpoint, es, lr_plateau]
#callbacks_list = [checkpoint, es]

In [ ]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks_list,
    class_weight=weights
)

In [ ]:
filepath = "saved_models/all_VGG16_models/best_model_weighted_RESNET50.keras"
best_model = tf.keras.models.load_model(filepath)

In [ ]:
scores = best_model.predict(test_ds)

In [ ]:
#acc = history.history['accuracy']
#val_acc = history.history['val_accuracy']

#loss = history.history['loss']
#val_loss = history.history['val_loss']

In [ ]:
#EPOCHS = 113

#plt.figure(figsize=(8, 8))
#plt.subplot(1, 2, 1)
#plt.plot(range(EPOCHS), acc, label='Training Accuracy')
#plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
#plt.legend(loc='lower right')
#plt.title('Training and Validation Accuracy')

#plt.subplot(1, 2, 2)
#plt.plot(range(EPOCHS), loss, label='Training Loss')
#plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
#plt.legend(loc='upper right')
#plt.title('Training and Validation Loss')
#plt.show()

In [ ]:
def create_list_y(dataset, model):
  y_true = []
  y_pred = []

  for x, y in dataset:
    y_true.append(y)
    y_pred.append(tf.argmax(model.predict(x),axis = 1))
    
  y_pred = tf.concat(y_pred, axis=0)
  y_true = tf.concat(y_true, axis=0)

  return y_pred, y_true

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

y_pred, y_true = create_list_y(test_ds, best_model)

cm = confusion_matrix(y_true, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=CLASS_NAMES)

disp.plot(cmap=plt.cm.Blues)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, target_names=CLASS_NAMES))

In [ ]:
from io import BytesIO
from PIL import Image

def read_file_as_img(file_path)-> np.ndarray:
    with open(file_path, 'rb') as f:
        data = f.read()
    image = np.array(Image.open(BytesIO(data)))

    return image

In [ ]:
# Load the tensorflow model
PROD_MODEL_PATH = "saved_models/all_VGG16_models/best_model_weighted_RESNET50.keras"

PROD_MODEL = tf.keras.models.load_model(PROD_MODEL_PATH)

In [ ]:
img = read_file_as_img("images/taylor-swift-eras-tour-032023-3-1-cab011d6ac7243e68b93d383134ad604.jpg")

# Resize the image to 256x256
img = tf.image.resize(img, [256, 256]) 

# Predict the image classification
img_batch = np.expand_dims(img, 0)
predictions = PROD_MODEL.predict(img_batch)

predicted_class = CLASS_NAMES[np.argmax(predictions[0])]

In [ ]:
predicted_class